# 뉴스 데이터프레임(감성 확률 포함) + 주가 데이터프레임 merge

In [1]:
import pandas as pd
import yfinance as yf

- 21년 1월 1일은 공휴일이므로 주가 데이터가 없음
- 매년 12월 31일은 휴장일임
- 따라서 20년 12월 30일(목)의 주가 데이터를 가져와 1월 1일을 채움

In [9]:
samsung = yf.Ticker("005930.KS")
hist = samsung.history(start='2021-01-01', end='2025-05-12', auto_adjust=False).reset_index() # end 미포함

# 컬럼 소문자 처리
hist.columns = hist.columns.str.lower()

# date 형식 변경
hist['date'] = pd.to_datetime(hist['date'])
hist['date'] = hist['date'].dt.date.astype(str)

# OHLCV 데이터만 가져오기
hist = hist[['date', 'open', 'high', 'low', 'close', 'volume']]

len(hist)

1065

In [10]:
prep_date = "250519"
today = "250520"

### 데이터 merge
- 주말 포함 : merged_weekend

In [ ]:
# 오늘 수집한 데이터 "전날"의 csv 불러오기
prep = pd.read_csv(f"{prep_date}_weekend_sentiment_stock.csv")

# --------------------------
# sentiment, stock 불러오기 ("오늘 수집")
sentiment = pd.read_csv(f"{today}_sentiment_daily.csv")
stock = pd.read_csv(f"{today}_stock.csv")

# 날짜 형식 정리
sentiment['date'] = pd.to_datetime(sentiment['date']).dt.normalize()
sentiment['date'] = sentiment['date'].dt.strftime('%Y-%m-%d')

stock['date'] = pd.to_datetime(stock['date']).dt.normalize()
stock['date'] = stock['date'].dt.strftime('%Y-%m-%d')

# --------------------------
# weekend 포함 : sentiment + stock merge
merged_weekend = pd.merge(sentiment, stock, on='date', how='left')

# --------------------------
# 준비된 12/30 행과 merged를 concat
merged_weekend = pd.concat([prep, merged_weekend], ignore_index=True)

# --------------------------
# 정렬 후 forward fill
merged_weekend = merged_weekend.sort_values('date').reset_index(drop=True)
merged_weekend[['open', 'high', 'low', 'close', 'volume']] = merged_weekend[
    ['open', 'high', 'low', 'close', 'volume']
].ffill()

merged_weekend.tail(2)

,date,1,2,3,4,5,6,7,8,9,...,news_count,avg_negative,avg_neutral,avg_positive,open,high,low,close,volume,label
1599,2025-05-19,"용석우 삼성전자 사장, 전자정보통신산업진흥회장 취임 ||| 용석우 제21대 한국전자...","삼성전자 OLED TV, 엔비디아 지싱크 호환 인증 ||| 삼성전자(005930)는...",[주식 초고수는 지금]'6만 전자' 기대감…삼성전자 순매수 1위 ||| 미래에셋증권...,"용석우 삼성전자 사장, KEA 회장 취임 “AI·공급망 대응 주력” ||| 공급망·...",[충무로에서] 화웨이·삼성전자의 같지만 다른 35조원 ||| '대륙의 실수' 기업 ...,전자정보통신산업진흥회장에 용석우 삼성전자 사장 취임 ||| 용석우 삼성전자 영상디스...,NaN,NaN,NaN,...,6,0.0528,0.6087,0.3385,56400.0,56400.0,55500.0,55800.0,9802105.0,0.0
1600,2025-05-20,"“가격 최대 30% 올리겠습니다”...TSMC 결정에 미소짓는 삼성전자, 왜? ||...","삼성전자, 전략 리더십 재정비 AI 중심 ‘뉴삼성’ 본격 시동 ||| 삼성전자가 다...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0.1680,0.7315,0.1010,56200.0,56700.0,55700.0,55900.0,9080577.0,NaN


- 주말 미포함 : merged_weekday

In [63]:
# 오늘 수집한 데이터 주말 포함 데이터프레임에서 주가 데이터 제거
sentiment_weekday = merged_weekend.drop(['open', 'high', 'low', 'close', 'volume', 'label'], axis=1)

# 주말, 공휴일 미포함 주가 데이터프레임
samsung = yf.Ticker("005930.KS")
hist = samsung.history(start='2021-01-01', end='2025-05-21', auto_adjust=False).reset_index() # end 미포함

# 컬럼 소문자 처리
hist.columns = hist.columns.str.lower()

# date 형식 변경
hist['date'] = pd.to_datetime(hist['date'])
hist['date'] = hist['date'].dt.date.astype(str)

# OHLCV 데이터만 가져오기
hist = hist[['date', 'open', 'high', 'low', 'close', 'volume']]

# 날짜 형식 정리
sentiment_weekday['date'] = pd.to_datetime(sentiment_weekday['date']).dt.normalize()
sentiment_weekday['date'] = sentiment_weekday['date'].dt.strftime('%Y-%m-%d')

hist['date'] = pd.to_datetime(hist['date']).dt.normalize()
hist['date'] = hist['date'].dt.strftime('%Y-%m-%d')

# inner join으로 sentiment와 stock_valid 병합
merged_weekday = pd.merge(sentiment_weekday, hist, on='date', how='inner')

merged_weekday.tail(2)

,date,1,2,3,4,5,6,7,8,9,...,39,news_count,avg_negative,avg_neutral,avg_positive,open,high,low,close,volume
1070,2025-05-19,"용석우 삼성전자 사장, 전자정보통신산업진흥회장 취임 ||| 용석우 제21대 한국전자...","삼성전자 OLED TV, 엔비디아 지싱크 호환 인증 ||| 삼성전자(005930)는...",[주식 초고수는 지금]'6만 전자' 기대감…삼성전자 순매수 1위 ||| 미래에셋증권...,"용석우 삼성전자 사장, KEA 회장 취임 “AI·공급망 대응 주력” ||| 공급망·...",[충무로에서] 화웨이·삼성전자의 같지만 다른 35조원 ||| '대륙의 실수' 기업 ...,전자정보통신산업진흥회장에 용석우 삼성전자 사장 취임 ||| 용석우 삼성전자 영상디스...,NaN,NaN,NaN,...,NaN,6,0.0528,0.6087,0.3385,56400.0,56400.0,55500.0,55800.0,9802105
1071,2025-05-20,"“가격 최대 30% 올리겠습니다”...TSMC 결정에 미소짓는 삼성전자, 왜? ||...","삼성전자, 전략 리더십 재정비 AI 중심 ‘뉴삼성’ 본격 시동 ||| 삼성전자가 다...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,2,0.1680,0.7315,0.1010,56200.0,56700.0,55700.0,55900.0,9080577


# 레이블(target) 생성
- 예측 모델 목표 : 다음날 주가 등락 방향 예측
- target : 이전 날 대비 주가 등락 여부
    - 상승 : 1
    - 하락 or 유지 : 0

In [6]:
def target_column(df) :
    df = df.sort_values('date').reset_index(drop=True)

    # 다음날 종가 생성
    df['next_close'] = df['close'].shift(-1)

    # 레이블 생성 : 다음날 종가가 오늘보다 높으면 1, 아니면 0
    df['label'] = (df['next_close'] > df['close']).astype(int)

    # 필요없는 컬럼 삭제
    merged = df.drop(columns=['next_close'])

    return merged

In [ ]:
df_weekend = target_column(merged_weekend)

df_weekend.tail(2)

,date,news_count,1,2,3,4,5,6,7,8,...,50,avg_negative,avg_neutral,avg_positive,open,high,low,close,volume,label
352,2025-05-19,255.0,‘성장동력 HVAC’⋯ LG가 공들이던 이 사업 삼성도 참전 선언 ||| 삼성전자가...,‘용인 반도체 클러스터’ 공업용수 실시설계 본격화 ||| 경기도 용인시에 들어설 반...,"국민의힘 대선후보 김문수, “국회 및 수도권 행정기관, 세종 이전할 것” ||| “...","밸류업 1년, 블록딜 반짝 늘었지만…규제·관세로 올해 주춤[시그널INSIDE] ||...",중동 수출 따낸 엔비디아 “중국 시장도 막지 말고 먹어야” ||| ━ AI칩 수출 ...,"“AI 투자, 제조업 돕는 솔루션 보유 기업이 유망” ||| 남기문(사진) 스마일게...",듣는 오디오에서 ‘읽는’ 오디오로… 진화하는 요즘 스피커 ||| LG전자가 지난달 ...,반도체부터 우주·국방·로봇… AI가 바꿀 미래를 엿보다 |||,...,"[찐코노미] ""이번엔 6만전자 안착?""···삼성전자 주가에 영향을 미칠 호재는? |...",0.0771,0.5643,0.3585,56400.0,56400.0,55500.0,55800.0,9802105,1
353,2025-05-20,221.0,“더 과감하게 도전하자”던 이재용… 신수종 사업서 ‘퀀텀 점프’ 모색 ||| 삼성전...,"삼성전자 OLED TV, ‘지싱크 호환’ 인증 ||| 삼성전자&nbsp;2025년형...","[사설] 용인 반도체 클러스터, 용수·전기 확보가 핵심이다 |||","용석우 삼성전자 사장, 21대 전자정보통신산업진흥회장 취임 ||| 용석우 삼성전자 ...","최윤화 제엠제코 대표, 올해의 발명왕 선정 ||| 양면 방열 기술을 적용해 고성능 ...",젠슨황 “대만에 AI 슈퍼컴 구축하겠다” ||| 젠슨황 엔비디아 최고경영자(CEO)...,젠슨 황 ‘핵심 파트너’ 대만을 점찍었다…“최고 AI 인프라 협업” ||| 인공지능...,코스피 상장사 1분기 영업익 57조… 636곳 중 478곳 흑자 ||| 1분기 코스...,...,"""한국 주식 사자"" 외인 매수에 코스피 강세 출발…원전주 강세 ||| [머니투데이 ...",0.0605,0.7156,0.2239,56200.0,56700.0,55700.0,55900.0,9080577,0


In [67]:
df_weekday = target_column(merged_weekday)

df_weekday.tail(2)

,date,1,2,3,4,5,6,7,8,9,...,news_count,avg_negative,avg_neutral,avg_positive,open,high,low,close,volume,label
1070,2025-05-19,"용석우 삼성전자 사장, 전자정보통신산업진흥회장 취임 ||| 용석우 제21대 한국전자...","삼성전자 OLED TV, 엔비디아 지싱크 호환 인증 ||| 삼성전자(005930)는...",[주식 초고수는 지금]'6만 전자' 기대감…삼성전자 순매수 1위 ||| 미래에셋증권...,"용석우 삼성전자 사장, KEA 회장 취임 “AI·공급망 대응 주력” ||| 공급망·...",[충무로에서] 화웨이·삼성전자의 같지만 다른 35조원 ||| '대륙의 실수' 기업 ...,전자정보통신산업진흥회장에 용석우 삼성전자 사장 취임 ||| 용석우 삼성전자 영상디스...,NaN,NaN,NaN,...,6,0.0528,0.6087,0.3385,56400.0,56400.0,55500.0,55800.0,9802105,1
1071,2025-05-20,"“가격 최대 30% 올리겠습니다”...TSMC 결정에 미소짓는 삼성전자, 왜? ||...","삼성전자, 전략 리더십 재정비 AI 중심 ‘뉴삼성’ 본격 시동 ||| 삼성전자가 다...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,2,0.1680,0.7315,0.1010,56200.0,56700.0,55700.0,55900.0,9080577,0


In [ ]:
df_weekend.to_csv(f"/Users/taeheon/stock_price/data/{today}_weekend_sentiment_stock.csv", index=False)
df_weekday.to_csv(f"/Users/taeheon/stock_price/data/{today}_weekday_sentiment_stock.csv", index=False)